In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import dpsimpy
import re

# %matplotlib widget

## RLC circuit

In [ ]:
time_step = 0.00005
final_time = 1

In [ ]:
sim_name = "EMT_RLC_py_RC"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.Single)
n2 = dpsimpy.emt.SimNode("n2", dpsimpy.PhaseType.Single)
n3 = dpsimpy.emt.SimNode("n3", dpsimpy.PhaseType.Single)

# Components
vs = dpsimpy.emt.ph1.VoltageSource("vs")
vs.set_parameters(complex(10, 0), 50)

r = 10.0
res = dpsimpy.emt.ph1.Resistor("r", dpsimpy.LogLevel.debug)
res.set_parameters(r)
l = 0.01
ind = dpsimpy.emt.ph1.Inductor("l", dpsimpy.LogLevel.debug)
ind.set_parameters(l)
c = 0.002
cap = dpsimpy.emt.ph1.Capacitor("c", dpsimpy.LogLevel.debug)
cap.set_parameters(c)

# Topology
vs.connect([gnd, n1])
res.connect([n1, n2])
ind.connect([n2, n3])
cap.connect([n3, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2, n3], [vs, res, ind, cap])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("i_RC", "i_intf", res)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)
sim.run()

In [ ]:
sim_name = "EMT_RLC_py_explSSN"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.Single)

# Components
vs = dpsimpy.emt.ph1.VoltageSource("vs")
vs.set_parameters(complex(10, 0), 50)

rlc = dpsimpy.emt.ph1.SSNTypeV2T("rlc", dpsimpy.LogLevel.debug)
r = 10.0
l = 0.01
c = 0.002
A = [[0, 1.0 / c], [-1.0 / l, -r / l]]
B = [[0.0], [1.0 / l]]
C = [[0.0, 1.0]]
D = [[0.0]]
rlc.set_parameters(A, B, C, D)

# Topology
vs.connect([gnd, n1])
rlc.connect([n1, gnd])

sys = dpsimpy.SystemTopology(50, [n1], [vs, rlc])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("i_explSSN", "i_intf", rlc)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)
sim.run()

In [ ]:
sim_name = "EMT_RLC_py_genSSN"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.Single)

# Components
vs = dpsimpy.emt.ph1.VoltageSource("vs")
vs.set_parameters(complex(10, 0), 50)

rlc = dpsimpy.emt.ph1.Full_Serial_RLC("rlc", dpsimpy.LogLevel.debug)
r = 10.0
l = 0.01
c = 0.002
rlc.set_parameters(r, l, c)

# Topology
vs.connect([gnd, n1])
rlc.connect([n1, gnd])

sys = dpsimpy.SystemTopology(50, [n1], [vs, rlc])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("i_genSSN", "i_intf", rlc)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)
sim.run()

In [ ]:
model_name = "EMT_RLC_py"

path_RC = "logs/" + model_name + "_RC" + "/"
dpsim_result_file_RC = path_RC + model_name + "_RC" + ".csv"
ts_dpsim_RC = read_timeseries_csv(dpsim_result_file_RC)

path_explSSN = "logs/" + model_name + "_explSSN" + "/"
dpsim_result_file_explSSN = path_explSSN + model_name + "_explSSN" + ".csv"
ts_dpsim_explSSN = read_timeseries_csv(dpsim_result_file_explSSN)

path_genSSN = "logs/" + model_name + "_genSSN" + "/"
dpsim_result_file_genSSN = path_genSSN + model_name + "_genSSN" + ".csv"
ts_dpsim_genSSN = read_timeseries_csv(dpsim_result_file_genSSN)

### Plot voltages

In [ ]:
plt.figure()

# RC
var_names = ["v1"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_RC[var_name].time,
        ts_dpsim_RC[var_name].values,
        label=var_name + "_RC",
    )

# explicit SSN
var_names = ["v1"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_explSSN[var_name].time,
        ts_dpsim_explSSN[var_name].values,
        label=var_name + "_explSSN",
        linestyle="-.",
        markevery=(6, 9),
    )

# generalized SSN
var_names = ["v1"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_genSSN[var_name].time,
        ts_dpsim_genSSN[var_name].values,
        label=var_name + "_genSSN",
        linestyle=":",
        markevery=(3, 9),
    )

plt.legend()
plt.show()

### Plot currents

In [ ]:
plt.figure()

# RC
var_names = ["i_RC"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_RC[var_name].time,
        ts_dpsim_RC[var_name].values,
        label=var_name,
    )

# explicit SSN
var_names = ["i_explSSN"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_explSSN[var_name].time,
        ts_dpsim_explSSN[var_name].values,
        label=var_name,
        linestyle="-.",
        markevery=(6, 9),
    )

# generalized SSN
var_names = ["i_genSSN"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_genSSN[var_name].time,
        ts_dpsim_genSSN[var_name].values,
        label=var_name,
        linestyle=":",
        markevery=(3, 9),
    )

plt.legend()
plt.show()

### Assertion RC vs. explicit SSN vs. generalized SSN

In [ ]:
epsilon = 1e-12

max_abs_err_RC_explSSN = (
    np.absolute(ts_dpsim_RC["i_RC"].values - ts_dpsim_explSSN["i_explSSN"].values)
).max()
max_abs_err_RC_genSSN = (
    np.absolute(ts_dpsim_RC["i_RC"].values - ts_dpsim_genSSN["i_genSSN"].values)
).max()
max_abs_err_explSSN_genSSN = (
    np.absolute(
        ts_dpsim_explSSN["i_explSSN"].values - ts_dpsim_genSSN["i_genSSN"].values
    )
).max()

max_rel_err_RC_explSSN = (
    max_abs_err_RC_explSSN / (np.absolute(ts_dpsim_RC["i_RC"].values)).max()
)
max_rel_err_RC_genSSN = (
    max_abs_err_RC_genSSN / (np.absolute(ts_dpsim_RC["i_RC"].values)).max()
)
max_rel_err_explSSN_genSSN = (
    max_abs_err_explSSN_genSSN
    / (np.absolute(ts_dpsim_explSSN["i_explSSN"].values)).max()
)

print(
    "Maximum absolute Error for RC vs. expicit SSN: ",
    max_abs_err_RC_explSSN,
    " giving a maximum relative error of: ",
    max_rel_err_RC_explSSN,
)
print(
    "Maximum absolute Error for RC vs. generalized SSN: ",
    max_abs_err_RC_genSSN,
    " giving a maximum relative error of: ",
    max_rel_err_RC_genSSN,
)
print(
    "Maximum absolute Error for explicit SSN vs. generalized SSN: ",
    max_abs_err_explSSN_genSSN,
    " giving a maximum relative error of: ",
    max_rel_err_explSSN_genSSN,
)

assert max_abs_err_RC_explSSN < epsilon
assert max_abs_err_RC_genSSN < epsilon
assert max_abs_err_explSSN_genSSN < epsilon

assert max_rel_err_RC_explSSN < epsilon
assert max_rel_err_RC_genSSN < epsilon
assert max_rel_err_explSSN_genSSN < epsilon